# Strategiset päätöksentekomallit

##Lineaarinen optimointi

Ratkaistaan seuraava optimointiongelma.

### Lähtötiedot

Yritys valmistaa kahta tuotetta, hilaa ja vitkutinta. Hilan kate on 250 euroa ja vitkuttimen kate 320 euroa tuotetta kohden. Valmistuksessa on neljä työvaihetta: leikkaus, stanssaus, puristus ja maalaus.

Kukin työvaihe vaatii tuotteilta seuraavasti aikaa (minuuttia):

<table align="center">
<tr>
<th></th>
<th>leikkaus</th>
<th>stanssaus</th>
<th>puristus</th>
<th>maalaus</th>
</tr>
  
<tr>
<th>hila</th>
<td>12</td>
<td>16</td>
<td>12</td>
<td>22</td>
</tr>
  
<tr>
<th>vitkutin</th>
<td>20</td>
<td>7</td>
<td>6</td>
<td>15</td>
</tr>

<tr>
<th>kapasiteetti</th>
<td>25&nbsp;000</td>
<td>30&nbsp;000</td>
<td>30&nbsp;000</td>
<td>20&nbsp;000</td>
</tr>
        
</table>

### Selvitettävät asiat

Selvitä oheisesta verkosta:

1. Optimaalinen tuotanto-ohjelma. (20 %)
2. Maksimikate. (20 %)
3. Yrityksellä on käytössään 25 000 euroa, jotka se voi käyttää minkä tahansa tuotantovaiheen mutta vain yhden tuotantovaiheen kapasiteetin lisäämiseen hintaan 1 €/min. Kuinka yrityksen kannattaisi käyttää 25 000 euroa kapasiteetin lisäykseen? (30 %)

Yrityksellä on lähtötilanteeseen liittyen (lisäkapasiteetti 25 000 euroa ei siis ole tässä tapauksessa käytössä) vielä tuote vilunki, jonka kate on 350 euroa tuotteelta ja joka vaatii kapasiteettia seuraavasti:

<table>
<tr>
<th></th>
<th>leikkaus</th>
<th>stanssaus</th>
<th>puristus</th>
<th>maalaus</th>
</tr>
   
<tr>
<th>vilunki</th>
<td>30</td>
<td>20</td>
<td>10</td>
<td>25</td>        
</tr>
</table>

4. Mikä on yrityksen optimituotanto ja maksimikate? (30 %)

## Ratkaisu

Otetaan käyttöön PuLP ja tuodaan kirjastot.

In [1]:
!pip install pulp

In [2]:
# Tuodaan kirjastot

from  pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

Optimointitehtävässä ratkaistaan lineaarinen optimointiongelma, jossa

Maksimoidaan
$$
250*hila +320*vitku
$$

Rajoitteina ovat

Leikkausrajoite: $$12*hila + 20*vitku \leq 25 000$$

Stanssausrajoite: $$16*hila + 7*vitku \leq 30 000$$

Puristusrajoite: $$12*hila + 6*vitku \leq 30 000$$

Maalausrajoite: $$22*hila + 15*vitku \leq 20 000$$

Luodaan malli.

In [3]:
# Luodaan malli

model = LpProblem(name="tuotanto-ongelma", sense=LpMaximize)

Alustetaan päätösmuuttujat.

In [4]:
# Alustetaan päätösmuuttujat

x = LpVariable(name="hila", lowBound=0, cat = "Integer")
y = LpVariable(name="vitkutin", lowBound=0, cat = "Integer")

Lisätään rajoitteet ja tavoitefunktio.

In [5]:
# Lisätään malliin rajoitteet

model += (12 * x + 20 * y <= 25000, "leikkausrajoite")
model += (16 * x + 7 * y <= 30000, "stanssausrajoite")
model += (12 * x + 6 * y <= 30000, "puristusrajoite")
model += (22 * x + 15 * y <= 20000, "maalausrajoite")


In [6]:
# Lisätään malliin tavoitefunktio

obj_func = 250 * x + 320 * y
model += obj_func

Katsotaan, mitä ollaan tähän asti tehty ja että kaikki on kunnossa.

In [7]:
model

tuotanto-ongelma:
MAXIMIZE
250*hila + 320*vitkutin + 0
SUBJECT TO
leikkausrajoite: 12 hila + 20 vitkutin <= 25000

stanssausrajoite: 16 hila + 7 vitkutin <= 30000

puristusrajoite: 12 hila + 6 vitkutin <= 30000

maalausrajoite: 22 hila + 15 vitkutin <= 20000

VARIABLES
0 <= hila Integer
0 <= vitkutin Integer

Nyt ollaan valmiita ratkaisemaan ongelma.

In [8]:
# Ratkaistaan optimointiongelma

status = model.solve()

Tutustutaan ratkaisuun.

In [9]:
# Optimointiratkaisun tilanne

print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


Optimiratkaisu siis löytyi. Maksimaalinen kate on 405 510 euroa:



In [10]:
# Optimointiongelman tavoiteratkaisu

print(f"objective: {model.objective.value()}")

objective: 405510.0


Se saadaan muuttujien hila ja vitkutin arvoilla hila = 95 ja vitkutin = 1 193.

In [11]:
# Päätösmuuttujien arvot

for var in model.variables():
    print(f"{var.name}: {var.value()}")

hila: 95.0
vitkutin: 1193.0


Ratkaisun rajoitteet ovat

In [12]:
# Rajoitteet

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

leikkausrajoite: 0.0
stanssausrajoite: -20129.0
puristusrajoite: -21702.0
maalausrajoite: -15.0


Käytetty ratkaisualgoritmi oli CBC.

In [13]:
# Ratkaisualgoritmi

model.solver

**Vastaus**:

* Optimaalinen tuotanto-ohjelma saadaan valmistamalla 95 kpl hiloja ja 1 193 kappaletta vitkuttimia.

* Maksmimikate on tuolloin 405 510 euroa.


### Kohta 3

Katsotaan, mihin tuotantovaiheesen lisäämällä päästään parhaaseen ratkaisuun.

In [14]:
# Luodaan malli

model = LpProblem(name="tuotanto-leikkaus+25000", sense=LpMaximize)

# Määritellään päätösmuuttujat

x = {i: LpVariable(name=f"x{i}", lowBound=0, cat = "Integer") for i in range(1, 3)}

# Lisätään rajoitteet

model += (12 * x[1] + 20 * x[2] <= 50000, "leikkausrajoite")
model += (16 * x[1] + 7 * x[2] <= 30000, "stanssausrajoite")
model += (12 * x[1] + 6 * x[2] <= 30000, "puristusrajoite")
model += (22 * x[1] + 15 * x[2] <= 20000, "maalausrajoite")

# Maäritetään tavoitefunktio

model += 250 * x[1] + 320 * x[2]

# Ratkaistaan optimointiongelma

status = model.solve()

print(f"ratkaisun tilanne: {model.status}, {LpStatus[model.status]}")
print(f"tavoitefunktion optimiarvo: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

ratkaisun tilanne: 1, Optimal
tavoitefunktion optimiarvo: 426560.0
x1: 0.0
x2: 1333.0
leikkausrajoite: -23340.0
stanssausrajoite: -20669.0
puristusrajoite: -22002.0
maalausrajoite: -5.0


In [15]:
# Luodaan malli

model = LpProblem(name="tuotanto-stanssaus+25000", sense=LpMaximize)

# Määritellään päätösmuuttujat

x = {i: LpVariable(name=f"x{i}", lowBound=0, cat = "Integer") for i in range(1, 3)}

# Lisätään rajoitteet

model += (12 * x[1] + 20 * x[2] <= 25000, "leikkausrajoite")
model += (16 * x[1] + 7 * x[2] <= 55000, "stanssausrajoite")
model += (12 * x[1] + 6 * x[2] <= 30000, "puristusrajoite")
model += (22 * x[1] + 15 * x[2] <= 20000, "maalausrajoite")


# Maäritetään tavoitefunktio

model += 250 * x[1] + 320 * x[2]

# Ratkaistaan optimointiongelma

status = model.solve()

print(f"ratkaisun tilanne: {model.status}, {LpStatus[model.status]}")
print(f"tavoitefunktion optimiarvo: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

ratkaisun tilanne: 1, Optimal
tavoitefunktion optimiarvo: 405510.0
x1: 95.0
x2: 1193.0
leikkausrajoite: 0.0
stanssausrajoite: -45129.0
puristusrajoite: -21702.0
maalausrajoite: -15.0


In [16]:
# Luodaan malli

model = LpProblem(name="tuotanto-puristus+25000", sense=LpMaximize)

# Määritellään päätösmuuttujat

x = {i: LpVariable(name=f"x{i}", lowBound=0, cat = "Integer") for i in range(1, 3)}

# Lisätään rajoitteet

model += (12 * x[1] + 20 * x[2] <= 25000, "leikkausrajoite")
model += (16 * x[1] + 7 * x[2] <= 30000, "stanssausrajoite")
model += (12 * x[1] + 6 * x[2] <= 55000, "puristusrajoite")
model += (22 * x[1] + 15 * x[2] <= 20000, "maalausrajoite")


# Maäritetään tavoitefunktio

model += 250 * x[1] + 320 * x[2]

# Ratkaistaan optimointiongelma

status = model.solve()

print(f"ratkaisun tilanne: {model.status}, {LpStatus[model.status]}")
print(f"tavoitefunktion optimiarvo: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

ratkaisun tilanne: 1, Optimal
tavoitefunktion optimiarvo: 405510.0
x1: 95.0
x2: 1193.0
leikkausrajoite: 0.0
stanssausrajoite: -20129.0
puristusrajoite: -46702.0
maalausrajoite: -15.0


In [17]:
# Luodaan malli

model = LpProblem(name="tuotanto-maalaus+25000", sense=LpMaximize)

# Määritellään päätösmuuttujat

x = {i: LpVariable(name=f"x{i}", lowBound=0, cat = "Integer") for i in range(1, 3)}

# Lisätään rajoitteet

model += (12 * x[1] + 20 * x[2] <= 25000, "leikkausrajoite")
model += (16 * x[1] + 7 * x[2] <= 30000, "stanssausrajoite")
model += (12 * x[1] + 6 * x[2] <= 30000, "puristusrajoite")
model += (22 * x[1] + 15 * x[2] <= 45000, "maalausrajoite")


# Maäritetään tavoitefunktio

model += 250 * x[1] + 320 * x[2]

# Ratkaistaan optimointiongelma

status = model.solve()

print(f"ratkaisun tilanne: {model.status}, {LpStatus[model.status]}")
print(f"tavoitefunktion optimiarvo: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

ratkaisun tilanne: 1, Optimal
tavoitefunktion optimiarvo: 504400.0
x1: 1800.0
x2: 170.0
leikkausrajoite: 0.0
stanssausrajoite: -10.0
puristusrajoite: -7380.0
maalausrajoite: -2850.0


In [18]:
# Lasketaan vertaamalla aiempaan, hyödytäänkö.

hyöty = 504400 - 405510

print(hyöty)

98890


Hyödyn suuruus on yli 25 000 euroa, joka oli lisäkapasiteetin kustannus, joten se kannattaa.

Otetaan tuotantoon vielä vilunki.

In [19]:
# Luodaan malli

model = LpProblem(name="tuotanto+vilunki", sense=LpMaximize)

# Määritellään päätösmuuttujat

x = {i: LpVariable(name=f"x{i}", lowBound=0, cat = "Integer") for i in range(1, 4)}

# x[1] on hila, x[1] vitkutin, x[2] vilunki

# Lisätään rajoitteet

model += (12 * x[1] + 20 * x[2] + 30 * x[3] <= 25000, "leikkausrajoite")
model += (16 * x[1] + 7 * x[2] + 20 * x[3] <= 30000, "stanssausrajoite")
model += (12 * x[1] + 6 * x[2] + 10 * x[3] <= 30000, "puristusrajoite")
model += (22 * x[1] + 15 * x[2] + 25 * x[3] <= 20000, "maalausrajoite")


# Maäritetään tavoitefunktio

model += 250 * x[1] + 320 * x[2] + 350 * x[3]

# Ratkaistaan optimointiongelma

status = model.solve()

print(f"ratkaisun tilanne: {model.status}, {LpStatus[model.status]}")
print(f"tavoitefunktion optimiarvo: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

ratkaisun tilanne: 1, Optimal
tavoitefunktion optimiarvo: 405510.0
x1: 95.0
x2: 1193.0
x3: 0.0
leikkausrajoite: 0.0
stanssausrajoite: -20129.0
puristusrajoite: -21702.0
maalausrajoite: -15.0


**Vastaus**: Tuotteen _vilunki_ mukaan ottaminen tuotantoon ei muuta optimituotantoa eikä maksimikatetta.

* Maksimikate: 405 510 euroa
* Hilojen lkm: 95 kappaletta
* Vitkuttimien lkm: 1 193 kappaletta.
* Vilunkien lkm: nolla kappaletta.

In [20]:
import datetime
print(f'Last modified {datetime.datetime.now():%Y-%m-%d %H:%M} by Juha Nurmonen')

Last modified 2024-09-19 13:40 by Juha Nurmonen
